In [2]:
import pandas as pd
import os
import json
import sys
import time
import traceback
import numpy as np
import os 


os.chdir(r'F:\360MoveData\Users\Administrator\Desktop')
df = pd.read_excel(r'data.xlsx',sheet_name=0)

def fun(x):
    m=[]
    if x:     
        for i in x:
            if  isinstance(i, dict):
                temp=i.get('val')
                if temp:
                    for j in temp:
                         if  isinstance(j, dict):
                            m.append(j.get('realLabelValue'))
        return m   
    else:
        return []

def fun1(x):
    if x:        
        for i in x:
            if  isinstance(i, dict):
                temp=i.get('shapeType')
        return temp
    else:
        return None

def fun2(x):
    m=[]
    if x:        
        for i in x:
            if  isinstance(i, dict):
                m.append(i.get('points'))
        return m
    else:
        return None

def fun3(x):
    m=[]
    if x:        
        for i in x:
            if  isinstance(i, dict):
                m.append(i.get('info'))
        return m
    else:
        return None 


def fun4(x):
    m=[]
    if x:     
        for i in x:
            for g in i:
                if  isinstance(g, dict):
                    temp=g.get('val')
                    if temp:
                        for j in temp:
                             if  isinstance(j, dict):
                                m.append(j.get('realLabelValue'))
        return m   
    else:
        return []

def formatrow(row, s_column, r_columns):
        s = row[s_column]
        if isinstance(s, dict):
            return [s.get(r_column) for r_column in r_columns]
        else :
            return [None]*len(r_columns)   


result = []
colum_dest = None
for i, row in df.iterrows():
    colums = row.index.values
    d = {}
    for colum in colums:
        try:
            json_o = json.loads(row[colum])
            d[colum] = json_o
            label=json_o['type']
            colum_dest = colum
        except:
            d[colum] = row[colum]
    result.append(d)

df = pd.DataFrame(result)
df['data']=df[colum_dest].apply(lambda s: s.get('data') if isinstance(s, dict) else None)

if label=='TXT':
    lables = ['start', 'columnIndex', 'end','annotateType','info']
    df[lables] = df.apply(formatrow,axis=1,result_type="expand",args=('data', lables))

    df = df.explode('info')
    df['info'] = df['info'].apply(lambda s: s.get('val') if isinstance(s, dict) else None)

    df = df.explode('info')

    df['data-realLabelValue'] = df['info'].apply(lambda s: s.get('realLabelValue') if isinstance(s, dict) else None)

    df['info1'] = df[colum_dest].apply(lambda s: s.get('info') if isinstance(s, dict) else None)


    df['realLabelValue1'] = df['info1'].apply(lambda x: ','.join(fun(x)))
    df=df.join(df['realLabelValue1'].str.split(',',expand=True))

    df.drop(columns=[colum_dest,'data','info','info1','realLabelValue1'], inplace=True)
    df.columns = ['info-realLabelValue' + str(col)  if  str(col).isdigit() else col  for col in df.columns]



elif label=='PHOTO':
    df['shapeType'] = df['data'].apply(lambda x: fun1(x))
    df['points'] = df['data'].apply(lambda x: fun2(x))
    df = df.explode('points')

    df['info'] = df['data'].apply(lambda x:  fun3(x))

    
    df['realLabelValue'] = df['info'].apply(lambda x:  ','.join(fun4(x)))
    df=df.join(df['realLabelValue'].str.split(',',expand=True))
    df.columns = ['realLabelValue' + str(col) if  str(col).isdigit() else col  for col in df.columns]

    

    df['info1'] = df[colum_dest].apply(lambda s: s.get('info') if isinstance(s, dict) else None)

    df['realLabelValue9'] = df['info1'].apply(lambda x: ','.join(fun(x)))
    df=df.join(df['realLabelValue9'].str.split(',',expand=True))


    df.drop(columns=[colum_dest,'data','info','info1','realLabelValue','realLabelValue9'], inplace=True)
    df.columns = ['info-realLabelValue' + str(col)  if  str(col).isdigit() else col  for col in df.columns]


df.reset_index(drop=True, inplace=True)
df=df.iloc[df.astype(str).drop_duplicates().index]
df.reset_index(drop=True, inplace=True)

df

,标注人员,shapeType,points,realLabelValue0,realLabelValue1,realLabelValue2,realLabelValue3,realLabelValue4,realLabelValue5,info-realLabelValue0,info-realLabelValue1,info-realLabelValue2,info-realLabelValue3,info-realLabelValue4,info-realLabelValue5,info-realLabelValue6
0,王二,rect,"[{'x': 67.04252577319585, 'y': 22.768041237113...",真菌,哺乳动物,游戏,西方魔幻游戏,东方仙侠游戏,游戏（按题材）,4级标签1,美发护发,None,None,None,None,None
1,王二,rect,"[{'x': 183.32216494845355, 'y': 39.84407216494...",真菌,哺乳动物,游戏,西方魔幻游戏,东方仙侠游戏,游戏（按题材）,4级标签1,美发护发,None,None,None,None,None
2,王二,rect,"[{'x': 78.42654639175257, 'y': 449.66881443298...",真菌,哺乳动物,游戏,西方魔幻游戏,东方仙侠游戏,游戏（按题材）,4级标签1,美发护发,None,None,None,None,None
3,张三,polygon,"[{'x': 124.50396825396824, 'y': 99.20634920634...",哺乳动物,宫斗游戏,东方仙侠游戏,None,None,None,3级标签,4级标签1,3级标签2,4级标签4,4级标签5,4级标签7,厨具餐具
4,杨六,None,None,,None,None,None,None,None,,None,None,None,None,None,None
